In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [2]:
explainer_map = {'conceptx': 'ConceptX', 'aconceptx': 'AntonymConceptX', 'conceptx_r': 'ConceptX-R', 'conceptx_a': 'ConceptX-A', 'conceptshap': 'ConceptSHAP', 'tokenshap': 'TokenSHAP', 'random': 'Random'}
explainer_order = ["Random", "TokenSHAP", "ConceptSHAP", "ConceptX", 'AntonymConceptX', "ConceptX-R", "ConceptX-A"]
MODEL_NAMES = {"mistral-7b-it": "Mistral-7B-Instruct", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o-mini", "llama-3-3b": "Llama-3.2-3B"}

In [13]:
save_dir = "/cluster/home/kamara/conceptx"
seed = 0
dataset = "saladbench"
model_name = "mistral-7b-it"


In [17]:
folder_path = os.path.join(save_dir, f"results/safety/mistral-7b-it/saladbench/seed_0")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print('parts: ', parts)
        
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        seed_idx = dataset_idx + 2
        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)

parts:  ['safety', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '8', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '5', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '6', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '7', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '4', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '10', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '3', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '2', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '0', 'saladbench', 'mistral-7b-it', '0.csv']
parts:  ['safety', 'batch', '1', 'saladbench', 'mistral-7b-it', '0.csv']


In [18]:
def compute_asr_hs(df):
    results = {
        "initial": {
            "ASR": (df["asr_0"] == "unsafe").mean(),
            "HS": df["score_0"].mean()
        },
        "gpt4o-mini": {
            "ASR": (df["asr_label"] == "unsafe").mean(),
            "HS": df["score_label"].mean()
        }
    }

    # Create a summary DataFrame for display
    summary_df = pd.DataFrame(results).T
    summary_df.columns = ["ASR", "HS"]
    return summary_df

In [19]:
summary = compute_asr_hs(df_all)
print(summary)

                 ASR        HS
initial     0.419162  2.384232
gpt4o-mini  0.119760  1.439122
